## Societyデータを大阪府の地図にマッピング
Societyで以前マッピングに利用したデータを使ってマッピングできたので、他の項目も追加したcsvを可視化してみることにした。</br>
https://soan.jp/plotly

In [9]:
import json
import re
import pandas as pd
import plotly.express as px
import geocoder as geocoder

# 定数
PREFECTURE = "大阪府"
CSV_FILE = "./data/area_mappingdata.csv"
GEOJSON_FILE = "./data/N03-20_27_200101.geojson"
MAPBOX_STYLE = "carto-positron"
COLOR_RANGE = (40, 75)
COLOR_SET = "気分_合計スコア"
ZOOM_LEVEL = 8
OPACITY = 0.5
FIG_SIZE = 800

#color_mapping = {
#    "1": "red",    
#    "2": "green",  
#    "3": "blue",
#    "4": "yellow"
#}

In [10]:
df = pd.read_csv(CSV_FILE, header=0)
df['気分_合計スコア'].describe() 

count    67.000000
mean     56.455224
std       6.661019
min      44.000000
25%      51.750000
50%      55.000000
75%      60.000000
max      71.000000
Name: 気分_合計スコア, dtype: float64

In [11]:
def load_data(CSV_FILE, geojson_file):
    """CSVファイルとGeoJSONファイルからデータフレームとGeoJSONを作成する"""
    # XLSファイルから人口データフレーム作成
    df = pd.read_csv(CSV_FILE, header=0)
    #大阪府のみ抽出
    #df = df[df['area'].str.startswith('27')]
    # クラスを数値型に変換する
    #df[['clus']] = df[['clus']].replace(',', '', regex=True).astype('float')

    # GeoJSONファイルから地図データ読み込み
    with open(geojson_file) as f:
        geojson = json.load(f)
        # GeoJSONデータから不要な列を削除する
        for feature in geojson['features']:
            feature['properties'].pop('N03_001')
            feature['properties'].pop('N03_002')
            feature['properties'].pop('N03_003')
            feature['properties'].pop('N03_004')

    return df, geojson

def plot_map(df, geojson, prefecture):
    """データフレームとGeoJSONから地図をプロットする"""
    # 都道府県の中心座標を取得
    center_latlng = geocoder.arcgis(prefecture).latlng

    # コロプレスマップを描画
    fig = px.choropleth_mapbox(
        df,
        geojson=geojson,
        locations="area",
        color=COLOR_SET,
#        color_discrete_map=color_mapping,  # 色を固定値で指定
#        hover_name="育児困難感_自責感",
#       hover_data=['人口','クラスター', '育児困難感_自責感'],
        featureidkey="properties.N03_007",
        mapbox_style=MAPBOX_STYLE,
        range_color=COLOR_RANGE,
        zoom=ZOOM_LEVEL,
        center={"lat": center_latlng[0]-0.5, "lon": center_latlng[1]},
        opacity=OPACITY,
        width=FIG_SIZE,
        height=FIG_SIZE,
    )
    fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
    fig.show()

In [ ]:
# メイン処理
if __name__ == "__main__":
    df_osaka, geojson_osaka = load_data(CSV_FILE, GEOJSON_FILE)
    plot_map(df_osaka, geojson_osaka, PREFECTURE)

In [17]:
df_osaka.dtypes

Unnamed: 0              int64
area                    int64
A_可住地面積１k.当たり人口密度     float64
A_昼夜間人口比率             float64
A_外国人人口.人口１０万人当たり.    float64
                       ...   
育児困難感_自信のなさ           float64
育児困難感_対処可能性           float64
育児困難感_子供への愛着          float64
育児困難感_自己肯定感           float64
育児困難感_自責感             float64
Length: 94, dtype: object

In [16]:
df_osaka

,Unnamed: 0,area,A_可住地面積１k.当たり人口密度,A_昼夜間人口比率,A_外国人人口.人口１０万人当たり.,A_１５歳未満人口割合,A_１５.６４歳人口割合,A_６５歳以上人口割合,A_未婚者割合.15歳以上人口.,A_人口増減率,...,育児困難感_感情コントロール,育児困難感_システム化衝動,育児困難感_同時・全体処理,育児困難感_同調困難,育児困難感_愛着の問題,育児困難感_自信のなさ,育児困難感_対処可能性,育児困難感_子供への愛着,育児困難感_自己肯定感,育児困難感_自責感
0,1,1101,8856.8,162.7,795.8,10.099,67.475,21.267,36.29654,7.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1102,4515.3,95.9,709.0,11.536,62.719,24.169,30.80597,2.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1103,4598.2,94.6,428.0,11.728,64.259,23.461,30.39551,2.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1104,6090.8,98.6,283.9,11.256,65.310,22.845,31.27262,2.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1105,7851.1,88.8,512.2,10.915,65.287,23.534,32.14211,3.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,1864,47361,194.0,100.5,438.4,15.706,57.705,26.460,27.68735,-8.97,...,3.0,2.75,3.75,2.0,1.0,2.75,1.75,1.0,2.0,2.25
1864,1865,47362,1163.1,86.0,206.4,18.286,61.969,19.301,30.53678,8.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1865,1866,47375,75.2,98.8,1172.5,17.420,56.198,26.382,24.44219,-3.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1866,1867,47381,70.7,105.0,950.5,17.009,61.756,19.960,31.14600,3.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_osaka.columns

Index(['Unnamed: 0', 'area', 'A_可住地面積１k.当たり人口密度', 'A_昼夜間人口比率',
       'A_外国人人口.人口１０万人当たり.', 'A_１５歳未満人口割合', 'A_１５.６４歳人口割合', 'A_６５歳以上人口割合',
       'A_未婚者割合.15歳以上人口.', 'A_人口増減率', 'A_転入超過率', 'A_転入率', 'A_転出率', 'A_核家族世帯割合',
       'A_高齢夫婦のみの世帯の割合', 'A_高齢単身世帯の割合', 'B_総面積.都道府県面積に占める割合.', 'B_可住地面積割合',
       'C_商業年間商品販売額.卸売業.小売業..従業者１人当たり.', 'C_商業年間商品販売額.卸売業.小売業..事業所当たり.',
       'E_小学校数.可住地面積100k.当たり.', 'E_中学校数.可住地面積100k.当たり.',
       'E_高等学校数.可住地面積100k.当たり.', 'F_就業者比率', 'F_第１次産業就業者比率.対就業者.',
       'F_第２次産業就業者比率.対就業者.', 'F_第３次産業就業者比率.対就業者.', 'F_第２次産業及び第３次産業就業者比率.対就業者.',
       'F_完全失業率', 'F_他市区町村への通勤者比率', 'F_他市区町村からの通勤者比率', 'F_高齢就業者割合.６５歳以上.',
       'G_公民館数.人口100万人当たり.', 'G_図書館数.人口100万人当たり.', 'H_小売店数.人口千人当たり.',
       'H_飲食店数.人口千人当たり.', 'H_大型小売店数.人口10万人当たり.', 'H_百貨店.総合スーパー数.人口10万人当たり.',
       'I_一般病院数.人口10万人当たり.', 'I_一般診療所数.人口10万人当たり.',
       'J_介護老人福祉施設.65歳以上人口10万人当たり.', '都市コード', 'クラスター', '人口', '世帯数', '出生児数',
       '年齢', '出産回数', '子人数', '周産期うつ傾向', 'Time', '気分_怒り～敵意', '気分_混乱～当惑',
       '気

In [18]:
df = pd.read_csv(CSV_FILE, header=0)
df['気分_合計スコア'].describe() 

count    67.000000
mean     56.455224
std       6.661019
min      44.000000
25%      51.750000
50%      55.000000
75%      60.000000
max      71.000000
Name: 気分_合計スコア, dtype: float64

In [19]:
df['気分_合計スコア'].describe() 

count    67.000000
mean     56.455224
std       6.661019
min      44.000000
25%      51.750000
50%      55.000000
75%      60.000000
max      71.000000
Name: 気分_合計スコア, dtype: float64